![](images/mta_logo.png)

# Mastering Large-Scale Open Datasets: The MTA's Comprehensive Guide to Socrata Query Language

## Overview
Open data platforms like [Socrata](https://dev.socrata.com/) have become an invaluable resource for researchers, policymakers, and businesses to analyze and make data-driven decisions. In this guide, we'll explore how to use the Socrata SoQL (Socrata Query Language) and Python's `requests` library to fetch data from a Socrata API.


## What is SODA?
SODA stands for "Socrata Open Data API." It's a set of APIs provided by Socrata to access various open datasets hosted by them. With SODA, you can programmatically access a wealth of information provided by governments, NGOs, and other organizations.

## Locating Open Data and APIs
How to Find MTA Open Data
- On your browser Search for [data.ny.gov](https://data.ny.gov/)
- You can browse our public catalog of all MTA open datasets from the Open Data Portal.

How to Find Your API Endpoint
- On DataLens: If you are viewing a DataLens, click on the "API" button in the upper left of the page.
- On Socrata Dataset: Click on "Export" and then "API." You'll find the API endpoint under "API Access Endpoint."

![](images/API_retrieve.png)

## Filetring and Exporting MTA Subway Hourly Ridership using Socrata Server

Performing data transformations using SoQL queries on Socrata's servers is generally faster than doing it on your local machine. Socrata's servers are optimized for this kind of operation, reducing the amount of data transferred and increasing performance.

For Example:

**1.  Access the Dataset:**

Open the MTA Subway Hourly Ridership dataset in your web browser. This will lead you to the dataset's primer page, which contains essential information like the last updated date and its data dictionary. You'll also see a preview of the dataset here.

Click on the "View Data" button located at the top of the screen. This action will take you directly into the dataset. Depending on the date range, this dataset can contain millions of rows detailing hourly ridership for various MTA stations.

![](images/view_data.png)

**2. Apply a Filter to the Data:**

To filter the dataset, look for the "Filter" menu located in the lower left corner of the screen and click on it. Utilizing the filter function helps to narrow down the vast dataset into more manageable chunks.

![](images/filters.png)

Under the Filter menu, select "Transit Timestamp" and choose the "is between" option. Now enter two dates based on your specific data needs and press enter.

![](images/filters_is_between.png)

**3. Export the Filtered Data:**

The dataset will refresh to display only the subway ridership records that fall between the date range you specified.

To export this subset of data, click on the "Export" button located in the upper right corner of the screen. You'll have options to export in various formats such as CSV, JSON, or Excel based on your preference using either "Download File" or "API Endpoint".

![](images/apiendpoint_or_csv_export.png)

## Making API Requests with Python
Here is how you can fetch the data using Python's `requests` library:

In [ ]:
import requests
url = "https://data.ny.gov/resource/wujg-7c2s.geojson?$limit=1000000&$where=transit_timestamp >= '2023-01-01T00:00:00' AND transit_timestamp <= '2023-01-31T00:00:00'"
response = requests.get(url)
data = response.json()

<div class="alert alert-block alert-info">
 This example fetches Subway Hourly Ridership data and it queries out for January 2023. The data is returned in JSON format, although you could specify other formats like CSV if you prefer.

 <b>Note:</b> We set `$limit=1000000` because If you don't specify a limit and the dataset is large, you may only get the first few hundred records by default.
</div>


<b>Note:</b> If you are on a corporate network, you might need to define and use proxies to access the API. Your IT department should be able to provide proxies for you, should you need them.

To use proxies, simply change above code to:

In [ ]:
import requests
url = "https://data.ny.gov/resource/wujg-7c2s.geojson?$limit=1000000&$where=transit_timestamp >= '2023-01-01T00:00:00' AND transit_timestamp <= '2023-01-31T00:00:00'"
proxies = {
    "http": "insert your proxy",
    "https": "insert your proxy",
}
response = requests.get(url, proxies=proxies)
data = response.json()

## Read Data into Pandas Dataframe

In [ ]:
import pandas as pd

# If the data is in 'features' key (common for geojson format)
df = pd.json_normalize(data['features'])

df

## Paging and Throttling

- Paging: Sometimes Socrata API requests will return a large number of results. Rather than retrieve them all at once, which may affect your applicationâ€™s performance, you can use paging to retrieve the results in batches. Often an application will show the first few results, and then only load the next batch of results when the user has taken an action, such as clicking a Next button or scrolling to the bottom of a list. Paging is accomplished through two query parameters: `$limit` and `$offset`. Note that using paging is entirely optional: if you do not specify the `$limit` and `$offset` parameters, then again the defaults will be used.

- Throttling: Free-tier users have a shared pool of requests. For more requests, register for an application token for up to 1000 requests per rolling hour period.

## Building Simple Filters and Queries

### SoQL Queries
SoQL statements are broken into â€œparametersâ€ similar to clauses in SQL statements. Each clause can be expressed either directly as a URL parameter or as a SoQL statement. If a parameter is not specified, then the default is used. SoQL allows us to query the database in a SQL-like syntax, which means we can perform various operations like filtering `$where`, sorting `$order`, and aggregation `$group` directly at the API level. Make sure to add `&` after every statement. For instance:

In [ ]:
#Each of these are SoQL statements starting with `$`
$where=transit_timestamp >= '2023-01-01T00:00:00' AND transit_timestamp <= '2023-01-31T00:00:00'
$order=transit_timestamp ASC
$group=station_complex, transit_timestamp
$select=station_complex, transit_timestamp, sum(ridership) as total_ridership

After `$...` statement add `&` for example:

https://data.ny.gov/resource/wujg-7c2s.geojson?$limit=1000000&$where=transit_timestamp >= '2023-01-01T00:00:00' AND transit_timestamp <= '2023-01-31T00:00:00'&$order=transit_timestamp ASC&$group=station_complex, transit_timestamp&$select=station_complex, transit_timestamp, sum(ridership) as total_ridership

This query (which is a URL) will filter the dataset to include only January 2023, sort it by timestamp, group it by station name and timestamp, and aggregate the sum of ridership.

### Insights and Applications
Peak Hour Analysis: By grouping and sorting data by station and hour, we can identify which stations are most crowded during peak hours.

Trend Analysis: Tracking sum of ridership over specific intervals can help observe trends in transit usage, which could be seasonal or event-driven.

Operational Improvements: Identifying low-usage hours could help in optimizing operational costs like staffing and electricity.

Policy Decisions: Detailed usage patterns can influence public transit policies, including fare changes, service hours, and infrastructure development.

### Conclusion
Open data platforms like Socrata with powerful querying capabilities like SoQL allow us to analyze complex datasets with ease. New Yorkâ€™s hourly ridership data can provide a wealth of information for a range of applications, from operational optimization to public policy formulation.

### References
[Getting Started with SODA](https://dev.socrata.com/consumers/getting-started.html)

[SoQL Queries](https://dev.socrata.com/docs/queries/)